In [10]:
# Other than torch dependencies
! pip install GPUtil
! pip install pynvml
! pip install ankh
! pip install tqdm
! pip install biopython
NUMBER_OF_JUNK_PROTEINS = 1000

In [17]:
import torch
print("Pytorch " + torch.__version__)

import pandas as pd
import numpy as np
import time
from GPUtil import showUtilization as gpu_usage

import ankh

from torch import nn
from torch.utils.data import Dataset, DataLoader

from transformers import Trainer, TrainingArguments, EvalPrediction
from datasets import load_dataset

from sklearn import metrics
from scipy import stats
from functools import partial
from tqdm.auto import tqdm


# Create device agnostic code
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

Pytorch 1.13.1


### Load data from TSV


In [12]:
# Check if running in Google Colab to run dumb-colab speciefic code
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    COLAB = True
else:
    print('Not running on CoLab')
    COLAB=False


Not running on CoLab


In [13]:
if COLAB:
    # Mount GDrive for Colab
    from google.colab import drive
    drive.mount('/content/drive')
    # Navigate Colab
    %cd /content/drive/MyDrive/Colab Notebooks/esm
    %ls

In [14]:
prot_voc = pd.read_csv(f"prot_voc_{NUMBER_OF_JUNK_PROTEINS}.tsv", sep='\t')

# Model download

In [15]:
model, tokenizer = ankh.load_base_model()
model.eval()
with torch.no_grad():
  model.to(DEVICE)

Some weights of the model checkpoint at ElnaggarLab/ankh-base were not used when initializing T5EncoderModel: ['decoder.block.16.layer.2.DenseReluDense.wo.weight', 'decoder.block.20.layer.0.SelfAttention.k.weight', 'decoder.block.7.layer.2.DenseReluDense.wi_1.weight', 'decoder.block.4.layer.2.DenseReluDense.wo.weight', 'decoder.block.0.layer.0.SelfAttention.k.weight', 'decoder.block.23.layer.0.SelfAttention.k.weight', 'decoder.block.15.layer.2.DenseReluDense.wo.weight', 'decoder.block.15.layer.2.DenseReluDense.wi_0.weight', 'decoder.block.2.layer.0.layer_norm.weight', 'decoder.block.13.layer.1.EncDecAttention.q.weight', 'decoder.block.7.layer.1.EncDecAttention.k.weight', 'decoder.block.16.layer.2.layer_norm.weight', 'decoder.block.22.layer.0.SelfAttention.q.weight', 'decoder.block.19.layer.2.layer_norm.weight', 'decoder.block.20.layer.1.EncDecAttention.k.weight', 'decoder.block.14.layer.1.EncDecAttention.q.weight', 'decoder.block.18.layer.2.DenseReluDense.wi_0.weight', 'decoder.block.3

In [18]:
def cache_clear(): # In colab to clear GPU cache you need to wait some time after deleting tensor
    if COLAB:
        time.sleep(0.1)
    torch.cuda.empty_cache()

def gpu_util(): # To monitor how much more can we load GPU with data
    if DEVICE == "cuda":
        return torch.cuda.memory_reserved(DEVICE)/torch.cuda.get_device_properties(DEVICE).total_memory
    if DEVICE == "cpu":
        return 0


In [19]:
gpu_usage()
gpu_util()


| ID | GPU | MEM |
------------------
|  0 |  0% | 89% |


0

In [20]:
protein_sequences = list(prot_voc["Prot"].values)
tokens = tokenizer.batch_encode_plus(protein_sequences,
                                        add_special_tokens=True,
                                        padding=True,
                                        is_split_into_words=False,
                                        return_tensors="pt")
print(tokens['input_ids'].shape)

torch.Size([1000, 3133])


In [21]:
cache_clear()
print(torch.cuda.memory_reserved(DEVICE))
print(torch.cuda.get_device_properties(DEVICE))
import gc
gc.collect()
gpu_util()
gpu_usage()

ValueError: Expected a cuda device, but got: cpu

In [25]:
BATCH_SIZE = 8 # For A100 Collab
batched_tokens = torch.split(tokens['input_ids'], BATCH_SIZE)
batched_attention_masks = torch.split(tokens['attention_mask'], BATCH_SIZE)
batched_tokens[0].shape

torch.Size([1, 3133])

In [26]:
import sys
print(sys.getsizeof(batched_tokens[0]), "\t", sys.getsizeof(batched_tokens[0].storage()))
print(sys.getsizeof(tokens['input_ids'].storage))
print(batched_tokens[0].element_size() * batched_tokens[0].nelement())

72 	 25064048
64
25064


In [30]:
%mkdir batched_embs

In [32]:
gpu_usage()
with torch.no_grad():
    for idx, batch in enumerate(batched_tokens):
        inputs = batch.to(DEVICE)
        embeddings = model(input_ids=inputs)[0] # Most restricting moment. Model returns output with attention maps that clog the GPU Memory.
        output = embeddings[:, 0].cpu()
        output = output.numpy()
        np.savetxt(f"./batched_embs/batch{idx}.txt", output)
        print(f"{idx}/{len(batched_tokens)}\t GPU Usage:{gpu_util()}")
        del inputs
        del embeddings
        del output
        cache_clear()
        #exit()

gpu_usage()
gpu_util()
cache_clear()
gpu_usage()

| ID | GPU | MEM |
------------------
|  0 |  0% | 89% |
0/1000	 GPU Usage:0
1/1000	 GPU Usage:0
2/1000	 GPU Usage:0


KeyboardInterrupt: 

### If Out Of Memory error:

In [45]:
# In case of OOM
# Utilization from model alone should be ~25% (for GTX 1080Ti)
try:
  del inputs
except:
  pass
try:
  del batched_attention_masks
except:
    pass
try:
  del batch
except:
    pass
try:
  del batched_tokens
except:
    pass
try:
  del embeddings
except:
    pass
try:
  del output
except:
  pass
cache_clear()
print(torch.cuda.memory_reserved(DEVICE))
print(torch.cuda.get_device_properties(DEVICE))
import gc
gc.collect()
gpu_util()

3204448256
_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15101MB, multi_processor_count=40)


0.20235981608167894

In [46]:
# Clear model itself from GPU - last resort
try:
  del model
except:
  pass
cache_clear()
print(torch.cuda.memory_reserved(DEVICE))
print(torch.cuda.get_device_properties(DEVICE))
import gc
gpu_usage()
gc.collect()
gpu_util()

3204448256
_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15101MB, multi_processor_count=40)
| ID | GPU | MEM |
------------------
|  0 |  0% | 27% |


0.20235981608167894

In [47]:
# Test of GPU cache clearing

import torch
from GPUtil import showUtilization as gpu_usage

print("Initial GPU Usage")
gpu_usage()

tensorList = []
for x in range(10):
  tensorList.append(torch.randn(10000000,10).cuda())   # reduce the size of tensor if you are getting OOM



print("GPU Usage after allcoating a bunch of Tensors")
gpu_usage()

del tensorList

print("GPU Usage after deleting the Tensors")
gpu_usage()

print("GPU Usage after emptying the cache")
cache_clear()
gpu_usage()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 27% |
GPU Usage after allcoating a bunch of Tensors
| ID | GPU | MEM |
------------------
|  0 |  2% | 34% |
GPU Usage after deleting the Tensors
| ID | GPU | MEM |
------------------
|  0 |  2% | 34% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 | 15% |  9% |
